In [1]:
from data_access.twitter_reader import TwitterReader
from data_storage.db_manager import save_tweets, texts

In [18]:
texts = texts()

'A proposito di PA e dipendenti pubblici: ieri ho prenotato appuntamento alle @PosteNews a #Roma Prati. Oggi alle 18 precise hanno chiamato il mio numero e alle 18.10 avevo finito. Abberlino me dovevo portà la seggiolina come i turchi, file anche di 6 ore al freddo per strada.'

In [ ]:
import pandas as pd
import nltk

nltk.download('punkt')

In [ ]:
df = pd.read_csv('./data/tweets.csv')

In [ ]:
df = df['text']
df.to_csv(r'./data/text_only.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [ ]:
reader = PlaintextCorpusReader('./data/', '.*\.txt')

In [ ]:
reader.sents('text_only.txt')[1]